<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/Find_tuning_BS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore

In [3]:
import pickle
import joblib

######Load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class'

train 3 class

In [6]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

#### train 3 class  --- สร้างตารางเก็บ ชื่อภาพ และชื่อคลาส
ข้อมูลใน All_3class_dir รวมภาพทั้ง 3 คลาส รวม 451 ภาพ TB150, Normal151 และ CA150 (CA คือ lungcancer)

In [7]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

In [8]:
normal = []
for i in range(len(os.listdir(Normal151_dir))):
    img = os.listdir(Normal151_dir)[i]
    normal.append(img)
    
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [9]:
tuberculosis = []
for i in range(len(os.listdir(TB150_dir))):
    img = os.listdir(TB150_dir)[i]
    tuberculosis.append(img)

ttuberculosis = pd.DataFrame({'ImageName':[tuberculosis][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [10]:
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [11]:
data_train3 = pd.concat([tnormal, ttuberculosis, tlungcancer])
data_train3[-3:]

,ImageName,NameType,NumberType
147,ca26.jpg,Lungcancer,2
148,ca7.jpg,Lungcancer,2
149,ca8.jpg,Lungcancer,2


####  test 3 class จากเน็ต

In [ ]:
TB700_dir = os.path.join(data_dir,'test_model/Tuberculosis700')
Normal94_dir = os.path.join(data_dir,'test_model/NORMAL94')
CA150_dir = os.path.join(data_dir,'CA150')                  #อันเดิมที่ใช้เทรน

AllTest_2class = os.path.join(data_dir,'test_model/AllTest_2class')

In [ ]:
#สำหรับ test model
normal = []
for i in range(len(os.listdir(Normal94_dir))):
    img = os.listdir(Normal94_dir)[i]
    normal.append(img)
    
tnormal94 = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [ ]:
#สำหรับ test model
tb700 = []
for i in range(len(os.listdir(TB700_dir))):
    img = os.listdir(TB700_dir)[i]
    tb700.append(img)

ttb700 = pd.DataFrame({'ImageName':[tb700][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [ ]:
#สำหรับ test model ข้อม฿ลเดิม
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [ ]:
data_test2 = pd.concat([tnormal94[:30], ttb700[:30], tlungcancer[:10]])

In [ ]:
test_NorTB = []
for ImageName in tqdm(data_test2['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = AllTest_2class +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))
    test_NorTB.append(img)

100%|██████████| 70/70 [00:19<00:00,  3.60it/s]


In [ ]:
data_test = np.array(test_NorTB, dtype="float32") / 255.0

labels_test = np.array(data_test2[['NameType']] )        
labels_test.shape = (len(labels_test),) 

le3_Incep = LabelEncoder()                                                                     
labels_test = le3_Incep.fit_transform(labels_test)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_test = to_categorical(labels_test, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

## K-fold  -- InceptionV3

In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [01:32<00:00,  4.87it/s]


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

aug = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model3_Incep.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_Incep = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_Incep.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
kf3_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf3_Incep.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train3_Incep = data3_Incep[train]
    y_train3_Incep = labels3_Incep[train]
    x_test3_Incep = data3_Incep[test]
    y_test3_Incep = labels3_Incep[test]

    H3_Incep = model3_Incep.fit( x=aug.flow(x_train3_Incep, y_train3_Incep, batch_size=BS),
	                                            validation_data=(x_test3_Incep, y_test3_Incep),
	                                            steps_per_epoch=len(x_train3_Incep) // BS,
	                                            epochs=10,  callbacks = [callback])
    
    pred3_Incep = model3_Incep.predict(x_test3_Incep)
    
    oos_y.append(y_test3_Incep)
    oos_pred.append(pred3_Incep)  
    print(classification_report(y_test3_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred3_Incep.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/10
45/45 [==============================] - 103s 2s/step - loss: 0.6675 - accuracy: 0.3194 - val_loss: 0.6413 - val_accuracy: 0.3956
Epoch 2/10
45/45 [==============================] - 102s 2s/step - loss: 0.6375 - accuracy: 0.4111 - val_loss: 0.6231 - val_accuracy: 0.4835
Epoch 3/10
45/45 [==============================] - 97s 2s/step - loss: 0.6219 - accuracy: 0.4472 - val_loss: 0.6100 - val_accuracy: 0.5165
Epoch 4/10
45/45 [==============================] - 99s 2s/step - loss: 0.6082 - accuracy: 0.4806 - val_loss: 0.5993 - val_accuracy: 0.5495
Epoch 5/10
45/45 [==============================] - 98s 2s/step - loss: 0.5958 - accuracy: 0.5444 - val_loss: 0.5904 - val_accuracy: 0.5824
Epoch 6/10
45/45 [==============================] - 99s 2s/step - loss: 0.5829 - accuracy: 0.5611 - val_loss: 0.5768 - val_accuracy: 0.6593
Epoch 7/10
45/45 [==============================] - 98s 2s/step - loss: 0.5727 - accuracy: 0.6222 - val_loss: 0.5719 - val_accuracy: 0.5824
Epoch 8/10

## 1) Find Tuning

### -->ACC = 0.90

*   BS = 32
*   activation="softmax"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 15

##### Data

In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [01:58<00:00,  3.80it/s]


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

87924736/87910968 [==============================] - 0s 0us/step


311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_IncepF = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 32
EPOCHS = 50

aug = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF = KFold(5, shuffle=True, random_state=40) 

oos_yF = []
oos_predF = []
fold = 0

for train, test in kf_IncepF.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF = data3_Incep[train]
    ytrain_IncepF = labels3_Incep[train]
    xtest_IncepF = data3_Incep[test]
    ytest_IncepF = labels3_Incep[test]

    M_IncepF = model3_IncepF.fit( x=aug.flow(xtrain_IncepF, ytrain_IncepF, batch_size=BS),
	                                            validation_data=(xtest_IncepF, ytest_IncepF),
	                                            steps_per_epoch=len(xtrain_IncepF) // BS,
	                                            epochs=15,  callbacks = [callback])
    
    pred_IncepF = model3_IncepF.predict(xtest_IncepF)
    
    oos_yF.append(ytest_IncepF)
    oos_predF.append(pred_IncepF)  
    print(classification_report(ytest_IncepF.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/15
11/11 [==============================] - 212s 18s/step - loss: 0.5453 - accuracy: 0.5701 - val_loss: 0.8733 - val_accuracy: 0.4396
Epoch 2/15
11/11 [==============================] - 188s 17s/step - loss: 0.3283 - accuracy: 0.7957 - val_loss: 0.8059 - val_accuracy: 0.4725
Epoch 3/15
11/11 [==============================] - 188s 17s/step - loss: 0.2771 - accuracy: 0.8476 - val_loss: 0.7430 - val_accuracy: 0.5604
Epoch 4/15
11/11 [==============================] - 188s 17s/step - loss: 0.1564 - accuracy: 0.9238 - val_loss: 0.7924 - val_accuracy: 0.5385
Epoch 5/15
11/11 [==============================] - 188s 17s/step - loss: 0.1670 - accuracy: 0.9207 - val_loss: 0.8088 - val_accuracy: 0.5714
Epoch 6/15
11/11 [==============================] - 188s 17s/step - loss: 0.1128 - accuracy: 0.9421 - val_loss: 0.8733 - val_accuracy: 0.5714
Epoch 7/15
11/11 [==============================] - 188s 18s/step - loss: 0.1176 - accuracy: 0.9299 - val_loss: 1.0824 - val_accuracy: 0.527

In [ ]:
oos_yF = np.concatenate(oos_yF)
oos_predF = np.concatenate(oos_predF)

In [ ]:
score = metrics.accuracy_score(oos_yF.argmax(axis=1),	        #ค่า acc
                                             oos_predF.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_yF.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

Fold score (accuracy): 0.9046563192904656
              precision    recall  f1-score   support

  Lungcancer       1.00      0.82      0.90       150
      Normal       0.99      0.90      0.94       151
Tuberculosis       0.78      0.99      0.88       150

    accuracy                           0.90       451
   macro avg       0.92      0.90      0.91       451
weighted avg       0.92      0.90      0.91       451



In [ ]:
#ลองเซฟโมเดล
fold = 2
if metrics.accuracy_score( ytest_IncepF.argmax(axis=1),  pred_IncepF.argmax(axis=1) )>0.7:
    name = (f' model3_IncepF{fold}')
    joblib.dump(model3_IncepF, name)

INFO:tensorflow:Assets written to: ram://a5d1d51e-3da4-454e-81fb-752e887bd981/assets


## 1.2) Find Tuning

### -->ACC =  0.86 
### -->เวลา 7306.17 วินาที 

*   BS = 32
*   activation="sigmoid"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 15

##### Data

In [12]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [01:48<00:00,  4.16it/s]


In [13]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [14]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

87924736/87910968 [==============================] - 0s 0us/step


311

In [15]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [16]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model3_IncepF12 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF12.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [17]:
INIT_LR = 1e-1
BS32 = 32
EPOCHS = 50

aug12 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [18]:
kf_IncepF12 = KFold(5, shuffle=True, random_state=40) 

oos_yF12 = []
oos_predF12 = []
fold = 0

for train, test in kf_IncepF12.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF12 = data3_Incep[train]
    ytrain_IncepF12 = labels3_Incep[train]
    xtest_IncepF12 = data3_Incep[test]
    ytest_IncepF12 = labels3_Incep[test]

    M_IncepF12 = model3_IncepF12.fit( x=aug12.flow(xtrain_IncepF12, ytrain_IncepF12, batch_size=BS32),
	                                            validation_data=(xtest_IncepF12, ytest_IncepF12),
	                                            steps_per_epoch=len(xtrain_IncepF12) // BS32,
	                                            epochs=15,  callbacks = [callback])
    
    pred_IncepF12 = model3_IncepF12.predict(xtest_IncepF12)
    
    oos_yF12.append(ytest_IncepF12)
    oos_predF12.append(pred_IncepF12)  
    print(classification_report(ytest_IncepF12.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF12.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/15
11/11 [==============================] - 195s 17s/step - loss: 0.5603 - accuracy: 0.6159 - val_loss: 0.8161 - val_accuracy: 0.3956
Epoch 2/15
11/11 [==============================] - 182s 18s/step - loss: 0.3375 - accuracy: 0.7896 - val_loss: 0.8045 - val_accuracy: 0.4505
Epoch 3/15
11/11 [==============================] - 183s 17s/step - loss: 0.2700 - accuracy: 0.8598 - val_loss: 0.7342 - val_accuracy: 0.6044
Epoch 4/15
11/11 [==============================] - 184s 17s/step - loss: 0.1899 - accuracy: 0.9085 - val_loss: 0.8486 - val_accuracy: 0.5824
Epoch 5/15
11/11 [==============================] - 182s 17s/step - loss: 0.1314 - accuracy: 0.9482 - val_loss: 1.0472 - val_accuracy: 0.5495
Epoch 6/15
11/11 [==============================] - 181s 16s/step - loss: 0.1045 - accuracy: 0.9543 - val_loss: 1.0726 - val_accuracy: 0.5714
              precision    recall  f1-score   support

  Lungcancer       0.80      0.14      0.24        28
      Normal       0.56      1.

In [19]:
oos_yF12 = np.concatenate(oos_yF12)
oos_predF12 = np.concatenate(oos_predF12)

In [20]:
score = metrics.accuracy_score(oos_yF12.argmax(axis=1),	        #ค่า acc
                                             oos_predF12.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_yF12.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF12.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

Fold score (accuracy): 0.8625277161862528
              precision    recall  f1-score   support

  Lungcancer       0.99      0.69      0.82       150
      Normal       0.85      0.99      0.92       151
Tuberculosis       0.79      0.90      0.84       150

    accuracy                           0.86       451
   macro avg       0.88      0.86      0.86       451
weighted avg       0.88      0.86      0.86       451



## 2) Find Tuning  

###--> ACC = 0.76  
###--> ใช้เวลา 8173.251 วินาที = 136.22 นาที = 2.27 ชั่วโมง

*   BS = 16
*   activation="softmax"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 30


##### Data

In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [03:31<00:00,  2.13it/s]


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

87924736/87910968 [==============================] - 1s 0us/step


311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_IncepF2 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 16
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.15,				  #เลื่อนลง-ขึ้น ซ้ายขวา
		height_shift_range=0.15,
		shear_range=0.15, 						#ขยับมุมภาพ ซ้าย และ/หรือ ขวา ไปด้านข้าง เช่น จะกลายเป็นสี่เหลี่ยมด้านขนาน 
		horizontal_flip=True,
		fill_mode='constant')

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF2 = KFold(5, shuffle=True, random_state=40) 

oos_yF2 = []
oos_predF2 = []
fold = 0

for train, test in kf_IncepF2.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF2 = data3_Incep[train]
    ytrain_IncepF2 = labels3_Incep[train]
    xtest_IncepF2 = data3_Incep[test]
    ytest_IncepF2 = labels3_Incep[test]

    M_IncepF2 = model3_IncepF2.fit( x=aug2.flow(xtrain_IncepF2, ytrain_IncepF2, batch_size=BS),
	                                            validation_data=(xtest_IncepF2, ytest_IncepF2),
	                                            steps_per_epoch=len(xtrain_IncepF2) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_IncepF2 = model3_IncepF2.predict(xtest_IncepF2)

    oos_yF2.append(ytest_IncepF2)
    oos_predF2.append(pred_IncepF2) 

    score = metrics.accuracy_score(ytest_IncepF2.argmax(axis=1),	        #ค่า acc
                                               pred_IncepF2.argmax(axis=1) )
    print(f'Fold #{fold} score (accuracy): {score}')
    

#    if metrics.accuracy_score( ytest_IncepF2.argmax(axis=1),  pred_IncepF2.argmax(axis=1) )>0.9:
 #       name = (f' model3_IncepF2_Fold{fold}')
  #      joblib.dump(model3_IncepF2, name)

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 187s 8s/step - loss: 0.4612 - accuracy: 0.6686 - val_loss: 1.4591 - val_accuracy: 0.3736
Epoch 2/30
22/22 [==============================] - 175s 8s/step - loss: 0.3046 - accuracy: 0.8052 - val_loss: 1.3021 - val_accuracy: 0.4396
Epoch 3/30
22/22 [==============================] - 173s 8s/step - loss: 0.2600 - accuracy: 0.8314 - val_loss: 1.0278 - val_accuracy: 0.5055
Epoch 4/30
22/22 [==============================] - 174s 8s/step - loss: 0.2032 - accuracy: 0.8634 - val_loss: 1.0180 - val_accuracy: 0.5714
Epoch 5/30
22/22 [==============================] - 174s 8s/step - loss: 0.1327 - accuracy: 0.9331 - val_loss: 1.1071 - val_accuracy: 0.6044
Epoch 6/30
22/22 [==============================] - 174s 8s/step - loss: 0.1238 - accuracy: 0.9331 - val_loss: 0.7453 - val_accuracy: 0.6813
Epoch 7/30
22/22 [==============================] - 174s 8s/step - loss: 0.1349 - accuracy: 0.9390 - val_loss: 0.4770 - val_accuracy: 0.7692
Epoch

In [ ]:
oos_yF2 = np.concatenate(oos_yF2)
oos_predF2 = np.concatenate(oos_predF2)

In [ ]:
print(classification_report(oos_yF2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF2.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.99      0.59      0.74       150
      Normal       0.93      0.72      0.81       151
Tuberculosis       0.60      0.98      0.74       150

    accuracy                           0.76       451
   macro avg       0.84      0.76      0.76       451
weighted avg       0.84      0.76      0.76       451



## 3) Find Tuning 
###--> ACC = 81
###--> 5815.668 วินาที

*   BS = 8
*   activation="softmax"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 30







In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 83.92it/s] 


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_IncepF3 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF3.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 8

aug3 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF3 = KFold(5, shuffle=True, random_state=40) 

oos_yF3 = []
oos_predF3 = []
fold = 0

for train, test in kf_IncepF3.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF3 = data3_Incep[train]
    ytrain_IncepF3 = labels3_Incep[train]
    xtest_IncepF3 = data3_Incep[test]
    ytest_IncepF3 = labels3_Incep[test]

    M_IncepF3 = model3_IncepF3.fit( x=aug3.flow(xtrain_IncepF3, ytrain_IncepF3, batch_size=BS),
	                                            validation_data=(xtest_IncepF3, ytest_IncepF3),
	                                            steps_per_epoch=len(xtrain_IncepF3) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_IncepF3 = model3_IncepF3.predict(xtest_IncepF3)
    
    oos_yF3.append(ytest_IncepF3)
    oos_predF3.append(pred_IncepF3)  
    print(classification_report(ytest_IncepF3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF3.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/30
45/45 [==============================] - 195s 4s/step - loss: 0.4942 - accuracy: 0.6361 - val_loss: 0.7942 - val_accuracy: 0.4286
Epoch 2/30
45/45 [==============================] - 183s 4s/step - loss: 0.3243 - accuracy: 0.8000 - val_loss: 1.0859 - val_accuracy: 0.4835
Epoch 3/30
45/45 [==============================] - 183s 4s/step - loss: 0.2680 - accuracy: 0.8333 - val_loss: 1.0556 - val_accuracy: 0.5275
Epoch 4/30
45/45 [==============================] - 183s 4s/step - loss: 0.2613 - accuracy: 0.8694 - val_loss: 0.3897 - val_accuracy: 0.8022
Epoch 5/30
45/45 [==============================] - 183s 4s/step - loss: 0.2072 - accuracy: 0.8944 - val_loss: 0.8960 - val_accuracy: 0.6044
Epoch 6/30
45/45 [==============================] - 183s 4s/step - loss: 0.1478 - accuracy: 0.9083 - val_loss: 0.5644 - val_accuracy: 0.7692
Epoch 7/30
45/45 [==============================] - 188s 4s/step - loss: 0.1602 - accuracy: 0.9250 - val_loss: 1.2475 - val_accuracy: 0.6813
     

In [ ]:
oos_yF3 = np.concatenate(oos_yF3)
oos_predF3 = np.concatenate(oos_predF3)

In [ ]:
print(classification_report(oos_yF3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF3.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       1.00      0.54      0.70       150
      Normal       0.94      0.91      0.93       151
Tuberculosis       0.66      0.98      0.79       150

    accuracy                           0.81       451
   macro avg       0.87      0.81      0.81       451
weighted avg       0.87      0.81      0.81       451



## 3.1) Find Tuning  

###--> ACC = 0.72
###--> ใช้เวลา 6354.25  วินาที 

*   BS = 8
*   activation="sigmoid"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 30


##### Data

In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 86.56it/s] 


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model3_IncepF31 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF31.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 8

aug31 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF31 = KFold(5, shuffle=True, random_state=40) 

oos_yF31 = []
oos_predF31 = []
fold = 0

for train, test in kf_IncepF31.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF31 = data3_Incep[train]
    ytrain_IncepF31 = labels3_Incep[train]
    xtest_IncepF31 = data3_Incep[test]
    ytest_IncepF31 = labels3_Incep[test]

    M_IncepF31 = model3_IncepF31.fit( x=aug31.flow(xtrain_IncepF31, ytrain_IncepF31, batch_size=BS),
	                                            validation_data=(xtest_IncepF31, ytest_IncepF31),
	                                            steps_per_epoch=len(xtrain_IncepF31) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_IncepF31 = model3_IncepF31.predict(xtest_IncepF31)
    
    oos_yF31.append(ytest_IncepF31)
    oos_predF31.append(pred_IncepF31)  
    print(classification_report(ytest_IncepF31.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF31.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/30
45/45 [==============================] - 207s 4s/step - loss: 0.4663 - accuracy: 0.6417 - val_loss: 0.7496 - val_accuracy: 0.4725
Epoch 2/30
45/45 [==============================] - 188s 4s/step - loss: 0.3244 - accuracy: 0.7944 - val_loss: 0.5518 - val_accuracy: 0.5934
Epoch 3/30
45/45 [==============================] - 186s 4s/step - loss: 0.2740 - accuracy: 0.8250 - val_loss: 0.5254 - val_accuracy: 0.7033
Epoch 4/30
45/45 [==============================] - 187s 4s/step - loss: 0.1984 - accuracy: 0.8917 - val_loss: 0.3224 - val_accuracy: 0.8022
Epoch 5/30
45/45 [==============================] - 191s 4s/step - loss: 0.1850 - accuracy: 0.9056 - val_loss: 0.5997 - val_accuracy: 0.7253
Epoch 6/30
45/45 [==============================] - 191s 4s/step - loss: 0.1834 - accuracy: 0.8972 - val_loss: 0.7332 - val_accuracy: 0.6813
Epoch 7/30
45/45 [==============================] - 190s 4s/step - loss: 0.1598 - accuracy: 0.8944 - val_loss: 0.6397 - val_accuracy: 0.7802
     

In [ ]:
oos_yF31 = np.concatenate(oos_yF31)
oos_predF31 = np.concatenate(oos_predF31)

In [ ]:
print(classification_report(oos_yF31.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF31.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

## 4) Find Tuning 
###--> ACC =  76
###--> ใช้เวลา 4447.95 วินาที
*   BS = 1
*   activation="softmax"
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001
*   epoch 30







In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 86.58it/s] 


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_IncepF4 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF4.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 1

aug4 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF4 = KFold(5, shuffle=True, random_state=40) 

oos_yF4 = []
oos_predF4 = []
fold = 0

for train, test in kf_IncepF4.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF4 = data3_Incep[train]
    ytrain_IncepF4 = labels3_Incep[train]
    xtest_IncepF4 = data3_Incep[test]
    ytest_IncepF4 = labels3_Incep[test]

    M_IncepF4 = model3_IncepF4.fit( x=aug4.flow(xtrain_IncepF4, ytrain_IncepF4, batch_size=BS),
	                                            validation_data=(xtest_IncepF4, ytest_IncepF4),
	                                            steps_per_epoch=len(xtrain_IncepF4) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_IncepF4 = model3_IncepF4.predict(xtest_IncepF4)
    
    oos_yF4.append(ytest_IncepF4)
    oos_predF4.append(pred_IncepF4)  
    print(classification_report(ytest_IncepF4.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF4.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/30
360/360 [==============================] - 249s 665ms/step - loss: 0.5995 - accuracy: 0.4694 - val_loss: 0.8508 - val_accuracy: 0.5714
Epoch 2/30
360/360 [==============================] - 240s 666ms/step - loss: 0.4782 - accuracy: 0.5861 - val_loss: 0.5635 - val_accuracy: 0.6044
Epoch 3/30
360/360 [==============================] - 238s 661ms/step - loss: 0.4199 - accuracy: 0.6667 - val_loss: 0.6837 - val_accuracy: 0.6484
Epoch 4/30
360/360 [==============================] - 239s 663ms/step - loss: 0.3905 - accuracy: 0.6917 - val_loss: 0.6726 - val_accuracy: 0.7033
Epoch 5/30
360/360 [==============================] - 289s 802ms/step - loss: 0.3414 - accuracy: 0.7472 - val_loss: 0.4193 - val_accuracy: 0.7033
Epoch 6/30
360/360 [==============================] - 239s 663ms/step - loss: 0.3496 - accuracy: 0.7667 - val_loss: 0.4928 - val_accuracy: 0.7033
Epoch 7/30
360/360 [==============================] - 237s 658ms/step - loss: 0.2890 - accuracy: 0.8333 - val_loss: 

In [ ]:
oos_yF4 = np.concatenate(oos_yF4)
oos_predF4 = np.concatenate(oos_predF4)

In [ ]:
print(classification_report(oos_yF4.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF4.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.83      0.70      0.76       150
      Normal       0.88      0.79      0.83       151
Tuberculosis       0.63      0.79      0.70       150

    accuracy                           0.76       451
   macro avg       0.78      0.76      0.76       451
weighted avg       0.78      0.76      0.76       451

